# Portfolio Optimization with Scipy
This is an example of how to use **scipy.optimize.minimize** to perform portfolio optimization.  The problem we are trying to solve with Portfolio Optimization is how to determine the optimum asset mix to optimize certai portfolio objectives

*  Minimize Volitility
*  Maximize Returns
*  Maximize the Portfolio's Sharpe Ratio 


## A. Calculate Earnings for Assests in Portfolio

Create list of tickers symbols of assets in the portfolio to optimize.  Also, define the **start** date and **end** date to pull.

In [1]:
import datetime

tickers = sorted(['VWILX','VPMAX','VFORX','VWNEX','VIIIX','VEMPX','VGHAX','VWELX','VIPSX'])

start = datetime.datetime(2016, 8, 1)
end = datetime.datetime.today()

### Download Daily Data
We will use **pandas_datareader** to download the daily **Adjusted Close** price of the assets in the porfolio from Google.

In [2]:
import pandas as pd
import pandas_datareader.data as web
import fix_yahoo_finance as yf

yf.pdr_override()

TICKER_DATA = yf.download(tickers, start=start)
TICKER_DATA.Close.head()

C:\Users\rick\Anaconda3\lib\site-packages\fix_yahoo_finance\__init__.py:43: DeprecationWarning: 
    Auto-overriding of pandas_datareader's get_data_yahoo() is deprecated and will be removed in future versions.
    Use pdr_override() to explicitly override it.
  DeprecationWarning)


[*********************100%***********************]  9 of 9 downloaded

,VEMPX,VFORX,VGHAX,VIIIX,VIPSX,VPMAX,VWELX,VWILX,VWNEX
Date,,,,,,,,,
2016-08-01,168.089996,30.230000,92.050003,198.600006,13.67,108.510002,38.919998,69.580002,65.930000
2016-08-02,166.009995,30.059999,91.849998,197.350006,13.61,108.160004,38.759998,69.169998,65.349998
2016-08-03,167.279999,30.070000,91.980003,198.020004,13.64,108.199997,38.860001,68.849998,65.980003
2016-08-04,167.399994,30.150000,91.870003,198.110001,13.70,108.459999,38.889999,69.120003,66.029999
2016-08-05,169.089996,30.350000,91.589996,199.820007,13.66,109.620003,39.070000,69.790001,66.849998


### Define the Risk Free Rate
We will use the expected daily returns of the **S&P 500**

In [3]:
RISK_FREE_DATA = web.DataReader('^GSPC', 'yahoo', min(TICKER_DATA.Close.index)).Close
RISK_FREE_RATE = RISK_FREE_DATA.pct_change()[1:].mean() 
RISK_FREE_RATE

0.0006936891759715261

### Calculate Business Days for the past Year¶
As a proxy for tradig days

In [4]:
from datetime import timedelta
BDAYS = RISK_FREE_DATA.loc[end - timedelta(days=365):end].asfreq('B').count()
BDAYS

251

### Plot Prices and Returns

In [5]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Category20 as plt
output_notebook()

p = figure( title = 'Portfolio Asset Prices' , plot_width=910 , x_axis_type='datetime')
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'
p.toolbar.active_drag = None

for i, a in enumerate(tickers,0):
    p.line(x=pd.DataFrame(TICKER_DATA.Close).index, y=pd.DataFrame(TICKER_DATA.Close)[a], legend=a, color=plt[len(tickers)][i])

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

In [6]:
p = figure( title = 'Portfolio Normalized Returns' , plot_width=910 , x_axis_type='datetime')
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'
p.toolbar.active_drag = None

p.line(RISK_FREE_DATA.index, RISK_FREE_DATA / RISK_FREE_DATA.iloc[0] - 1 , 
       color='red', line_width=5, legend='S&P 500', alpha=0.25)

NORMALIZED_RETURNS = TICKER_DATA.Close / TICKER_DATA.Close.iloc[0] - 1
for i, a in enumerate(tickers,0):
    p.line(x=pd.DataFrame(NORMALIZED_RETURNS).index, y=pd.DataFrame(NORMALIZED_RETURNS)[a], legend=a, color=plt[len(tickers)][i])

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

### Calculate Daily Assets Returns
Determine the percent change from one day to the next, as daily return, and save it to the **RETURNS** data frame.

$$ \hbox{Expected Returns} = \frac{Close - Close_{-1} }{Close_{-1} } $$

In [7]:
RETURNS = TICKER_DATA['Close'].pct_change()[1:]
RETURNS.tail()

,VEMPX,VFORX,VGHAX,VIIIX,VIPSX,VPMAX,VWELX,VWILX,VWNEX
Date,,,,,,,,,
2018-01-08,0.002867,0.000819,-0.003700,0.001681,-0.001535,-0.002090,0.000235,0.004500,0.001360
2018-01-09,-0.000656,0.000273,0.006527,0.001598,-0.003075,0.001734,-0.000235,0.001792,0.001235
2018-01-10,-0.001501,-0.001364,-0.001006,-0.001117,0.000000,0.002163,0.000235,-0.004373,-0.000740
2018-01-11,0.014655,0.006284,0.006827,0.007108,-0.000771,0.008203,0.006105,0.003294,0.009752
2018-01-12,0.003009,0.005973,0.006114,0.006781,0.000000,0.010705,0.005368,0.013034,0.007090


In [8]:
p = figure( title = 'Asset Returns' , plot_width=910 , x_axis_type='datetime')
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Returns'
p.toolbar.active_drag = None

for i, a in enumerate(tickers,0):
    p.line(x=pd.DataFrame(RETURNS).index, y=pd.DataFrame(RETURNS)[a], legend=a, color=plt[len(tickers)][i])

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

### Calculate the average daily returns
For each asset in the portfolio, we need to determine the average returns by calculating the mean of the observed days.

In [9]:
EXPECTED_RETURNS = RETURNS.mean()
EXPECTED_RETURNS

VEMPX    0.000717
VFORX    0.000566
VGHAX   -0.000015
VIIIX    0.000683
VIPSX   -0.000141
VPMAX    0.000751
VWELX    0.000286
VWILX    0.001062
VWNEX    0.000629
dtype: float64

These returns can be annualized by $$ (( Daily Returns + 1)^{Business Days} - 1) * 100% $$

In [10]:
((EXPECTED_RETURNS + 1)**BDAYS -1)*100

VEMPX    19.709514
VFORX    15.264540
VGHAX    -0.385888
VIIIX    18.700967
VIPSX    -3.482361
VPMAX    20.731156
VWELX     7.448118
VWILX    30.516653
VWNEX    17.082315
dtype: float64

### Calculate the covariance matrix of the returns
The variance/covariance matrix provides a measure of volatility for the assets (and between assets) in the portfolio.

In [11]:
COVARIANCE = RETURNS.cov()
COVARIANCE

,VEMPX,VFORX,VGHAX,VIIIX,VIPSX,VPMAX,VWELX,VWILX,VWNEX
VEMPX,0.000047,2.473944e-05,0.000027,2.829220e-05,-1.560823e-06,0.000031,0.000017,0.000027,0.000033
VFORX,0.000025,2.029925e-05,0.000018,1.947618e-05,7.905497e-07,0.000021,0.000012,0.000023,0.000021
VGHAX,0.000027,1.833290e-05,0.000061,2.261186e-05,5.298660e-06,0.000034,0.000018,0.000026,0.000028
VIIIX,0.000028,1.947618e-05,0.000023,2.376601e-05,5.594818e-07,0.000026,0.000016,0.000023,0.000026
VIPSX,-0.000002,7.905497e-07,0.000005,5.594818e-07,8.988252e-06,0.000003,0.000005,0.000004,0.000001
VPMAX,0.000031,2.118580e-05,0.000034,2.629289e-05,3.220911e-06,0.000046,0.000018,0.000030,0.000037
VWELX,0.000017,1.226285e-05,0.000018,1.575175e-05,4.507826e-06,0.000018,0.000017,0.000015,0.000019
VWILX,0.000027,2.316445e-05,0.000026,2.314775e-05,3.544009e-06,0.000030,0.000015,0.000042,0.000025
VWNEX,0.000033,2.088237e-05,0.000028,2.595939e-05,1.299888e-06,0.000037,0.000019,0.000025,0.000039


## B. Create a coupe of utility functions


Define a function (**randomPortfolioWeights**) to generate a random mixed portfolio and a function (**PORTFOLIO**) to calculate the portfolio's **mean returns** ($\mu = w * p^T$) and **standard deviation** ($\sigma = \sqrt{w^T * C * w}$).  Where $w$ is a **vector of asset weights** and $C$ is a **covariance matrix**.


In [12]:
import numpy as np


def randomPortfolioWeights(ticker_count):
    ''' 
    Create a portfolio mix without shorts
    '''
    w = np.random.uniform(0,1,ticker_count)
    return w / sum(w)



def PORTFOLIO( weights ):
    ''' 
    Calculates and returns the portfolio 
        Avgerage Daily Returns (mu),  
        Standard deviation of the Daily Returns (sigma), and
        Sharpe Ratio
        Annualized Returns
    '''
    p = np.asmatrix(EXPECTED_RETURNS)
    w = np.asmatrix(weights)
    C = np.asmatrix(COVARIANCE)
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    sharpe = (mu - RISK_FREE_RATE)/sigma
    annualized_returns = ((mu[0,0] + 1)**BDAYS - 1) * 100
    
    return [mu[0,0], sigma[0,0], sharpe[0,0], annualized_returns]
    

Generate a set of **n_portfolios** portfolios, with random mixed parts of the assets in **tickers**, and use the **PORTFOLIO** function to obtain *Expected Daily Returns*, *Standard Deviation* of the Daily Returns, and the portfolio's *Sharpe ratio*.

In [13]:
n_portfolios = 10000
means, stds, sharpe, annu_returns = np.column_stack([
    PORTFOLIO( randomPortfolioWeights(len(tickers)) ) 
    for _ in range(n_portfolios)
])

### Calculate the Efficient Frontier
The Efficient Frontier is the line of maximum returns as a function of volatility for a mix of assets.

http://www.quantandfinancial.com/2013/07/mean-variance-portfolio-optimization.html

In [47]:
from scipy.optimize import minimize


def miniVolatilityFitness(W, r):
    '''For given level of return r, find weights which minimizes
    portfolio variance.'''
    mean, var, _, _ = PORTFOLIO(W)
    # Big penalty for not meeting stated portfolio return 
    # effectively serves as optimization constraint
    penalty = 1000 * abs(mean-r)       
    return var + penalty
    
def solveEfficientFrontier(R):
    '''
    Given an expected Return and Maximum Weight, calculate the 
    lowest volatility for the target Return.
    '''
    frontier_mean, frontier_var, frontier_sharpe, frontier_aret = [], [], [], []
    n = len(R)  # Number of assets in the portfolio
    W = np.ones([n])/n     # start optimization with equal weights
    bounds = [(0,1) for i in range(n)]
    constraints = ({'type':'eq', 'fun': lambda W: sum(W)-1. })
    
    # Iterate through the range of returns on Y axis
    for r in np.linspace(min(R), max(R), num=50): 
        optimized = minimize(fun=miniVolatilityFitness, x0=W, args=r, method='SLSQP', 
                             constraints=constraints, bounds=bounds, options={'ftol': 1e-9})   
        if not optimized.success: 
            raise BaseException(optimized.message)
            
        # add point to the min-var frontier [x,y] = [optimized.x, r]
        mu, sig, shp, aret = PORTFOLIO(optimized.x)
        frontier_mean.append(mu)
        frontier_var.append(sig)
        frontier_sharpe.append(shp)
        frontier_aret.append(aret)
        
    return frontier_mean, frontier_var, frontier_sharpe, frontier_aret

frontier_mean, frontier_var, frontier_sharpe, frontier_aret = solveEfficientFrontier(EXPECTED_RETURNS)

### Plot the Markowitz Bullet of Random Mixed Weights Portfolios

Use the expected returns and volatility of the random portfolios to draw the Markowitz Bullet of the porfolios.

In [49]:
p = figure( title = 'Markowitz Bullet for set of Random Portfolios' , plot_width=980 )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
p.toolbar.active_drag = None
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='gold')

show(p)

## D. Find the Minimum Volatility Portfolio

The **Scipy minimize** function will need a function with one objective (Variance).  The **getVolatility** function wraps the **PORTFOLIO** function and only returns the expected variance.

In [50]:
def getVolatility(mix):
    return( PORTFOLIO(mix)[1] )

In [51]:
from scipy.optimize import minimize

x0 = np.ones(len(tickers)) 


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1

OPT_VOLATILITY = minimize(getVolatility, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-8})


print( "Lowest Daily Volatility {}".format( np.round(OPT_VOLATILITY.fun, 4) ) )
OPT_VOLATILITY_MIX = pd.DataFrame(np.round( OPT_VOLATILITY.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_VOLATILITY_MIX[OPT_VOLATILITY_MIX['Percent of Portfolio']>0]

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.002555553097808215
            Iterations: 25
            Function evaluations: 275
            Gradient evaluations: 25
Lowest Daily Volatility 0.0026


,Percent of Portfolio
VFORX,20.77
VIIIX,8.81
VIPSX,70.42


In [52]:
min_volatility_mean , min_volatility_std , min_volatility_sharpe , min_volatility_annualized = PORTFOLIO( OPT_VOLATILITY.x ) 
print( 'Markowitz Bullet for Portfolio with the minimum volatility of {1}, daily returns of {0}, and Sharpe ratio of {2}'.\
           format(np.round(min_volatility_mean,4) , np.round(min_volatility_std,4) , np.round(min_volatility_sharpe,4) ) )

Markowitz Bullet for Portfolio with the minimum volatility of 0.0026, daily returns of 0.0001, and Sharpe ratio of -0.2408


In [53]:
p = figure( title = 'Portfolio with the minimum volatility of {1}, annualized returns of {0}%, and Sharpe ratio of {2}'.\
           format(np.round(min_volatility_annualized,2) , np.round(min_volatility_std,4) , np.round(min_volatility_sharpe,4) ) , 
           plot_width=980  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns' 
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.circle(x=min_volatility_std, y=min_volatility_mean, fill_alpha=0.77 , size=14 , color='red' )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='gold')
p.toolbar.active_drag = None
show(p)

## E. Efficient Portfolio for the highest daily returns

In [54]:
def getReturns(mix):
    return( -PORTFOLIO(mix)[0] )

In [55]:
from scipy.optimize import minimize

x0 = np.ones(len(tickers))  


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES BETWEEN -1 AND 1

OPT_RETURNS = minimize(getReturns, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-7})


print( "Highest Daily Returns {}".format( np.round(-OPT_RETURNS.fun, 4) ) )
OPT_RETURNS_MIX = pd.DataFrame(np.round( OPT_RETURNS.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_RETURNS_MIX[OPT_RETURNS_MIX['Percent of Portfolio']>0]

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.0010616414020425261
            Iterations: 35
            Function evaluations: 385
            Gradient evaluations: 35
Highest Daily Returns 0.0011


,Percent of Portfolio
VWILX,100.0


In [56]:
target_return_mean , target_return_std , target_return_sharpe , target_return_annualized = PORTFOLIO( OPT_RETURNS.x ) 
print( 'Markowitz Bullet for Portfolio with maximum daily returns of {0}, sigma of {1}, and Sharpe ratio {2}'.\
           format(np.round(target_return_mean,4) , np.round(target_return_std,4) , np.round(target_return_sharpe,5) ) )

Markowitz Bullet for Portfolio with maximum daily returns of 0.0011, sigma of 0.0065, and Sharpe ratio 0.05689


In [57]:
p = figure( title = 'Portfolio with maximum annual returns of {0}%, sigma of {1}, and Sharpe ratio {2}'.\
           format(np.round(target_return_annualized,2) , np.round(target_return_std,4) , np.round(target_return_sharpe,4) ) , 
           plot_width=980  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.circle(x=target_return_std, y=target_return_mean, fill_alpha=0.77 , size=14 , color='red' )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='gold')
p.toolbar.active_drag = None
show(p)

## F. Optimum Sharpe Ratio

http://www.investopedia.com/terms/s/sharperatio.asp

The **Scipy minimize** function will need a function with one objective (-Sharpe ratio).  The **getSharpe** function wraps the **PORTFOLIO** function and only returns the Sharpe ratio.

In [58]:
def getSharpe(mix):
    return( -PORTFOLIO(mix)[2] )

In [59]:
from scipy.optimize import minimize


x0 = np.ones(len(tickers)) 


cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})  # MIX SHOULD ADD TO ONE i.e. 'sum(x) = 1' -> 'sum(x) - 1 = 0'


bnds = tuple((0.0, 1.0) for x in x0)  # NO SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1
#bnds = tuple((-1.0, 1.0) for x in x0)  # ALLOW SHORT... ALL VALUES POSITIVE BETWEEN 0 AND 1

OPT_SHARPE = minimize(getSharpe, x0, method='SLSQP', bounds=bnds, constraints=cons, options={'disp': True, 'ftol': 1e-7})


print( "Highest Sharpe Ratio {}".format( np.round(-OPT_SHARPE.fun, 4) ) )
OPT_SHARPE_MIX = pd.DataFrame(np.round( OPT_SHARPE.x * 100, 2 ), index=RETURNS.columns, columns=['Percent of Portfolio'])
OPT_SHARPE_MIX[OPT_SHARPE_MIX['Percent of Portfolio']>0]

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.05689325290265349
            Iterations: 9
            Function evaluations: 99
            Gradient evaluations: 9
Highest Sharpe Ratio 0.0569


,Percent of Portfolio
VWILX,100.0


In [60]:
best_sharpe_return , best_sharpe_std , best_sharpe , best_sharpe_annualized_return = PORTFOLIO( OPT_SHARPE.x.T ) 
print('Markowitz Bullet for Portfolio with maximum Sharpe ratio of {2}, return of {0}, and sigma of {1}'.\
           format(np.round(best_sharpe_return,4) , np.round(best_sharpe_std,4) , np.round(best_sharpe,4)) )

Markowitz Bullet for Portfolio with maximum Sharpe ratio of 0.0569, return of 0.0011, and sigma of 0.0065


In [61]:
p = figure( title = 'Portfolio with maximum Sharpe ratio of {2}, annualized returns of {0}%, and sigma of {1}'.\
           format(np.round(best_sharpe_annualized_return,2) , np.round(best_sharpe_std,4) , np.round(best_sharpe,4) ) , 
           plot_width=980  )
p.xaxis.axis_label = 'Volatility'
p.yaxis.axis_label = 'Daily Returns'
 
p.circle(x=stds, y=means, fill_alpha=0.2, size=2 )
p.circle(x=best_sharpe_std, y=best_sharpe_return, fill_alpha=0.77 , size=14 , color='red' )
p.line(x=frontier_var, y=frontier_mean, line_width=2, color='gold')
p.toolbar.active_drag = None
show(p)